# CSCI 4253 / 5253 - Lab #3 - Patent Problem with SQL - SOLUTION
<div>
 <h2> CSCI 4283 / 5253 
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right"/> </h2>
</div>

In this assignment, we're going to solve a problem for which you'll also see the solution in Hadoop and then implement a PySpark solution. We have two databases
* One contains information about patents
* One contains information about patent citations (one patent citing the work of another)
The problem we're going to solve is augmenting the original patent data to include the number of *co-state citations*. In other words, if patent X was issued to someone in Colorado and patent Y was also issued to someone in Colorado and X cites Y, then this is a co-state citation.

The easiest way to solve this is to build an intermediary product from the patent citations table. That table contains `CITING` and `CITED` columns; you would augment it by adding `CITING_STATE` and `CITED_STATE`. Then, it becomes fairly simple to filter out all the cases where those states don't match. You can then use an sql `GROUP BY` to `COUNT(*)` all the co-state citations for a given patent and then join that with the original patents table resulting in an augmented table.

The final results for the first 13 words, sorted in descending order by co-state citations looks like the following
![this final output](final-output.png)

The challenge is going to be that I want you do to this *a single SQL query*. We're assuming you've learned some SQL in a previous life and if not, [now is a great time to learn](https://www.sqlitetutorial.net/). Even if you've done basic SQL, you'll probably need to review [using `select` in a where-clause or using multiple joins](https://dba.stackexchange.com/questions/33553/using-select-in-the-where-clause-of-another-select).


## Logistics

We're going to be using the SQLite3 system which runs entirely from a file (no server needed). The `Makefile` contains commands to download the raw data as ZIP files.

We can run shell commands in our notebook using [builtin "magic" commands](https://ipython.readthedocs.io/en/stable/interactive/magics.html). You need to run this at least once prior to starting the lab to make certain you have the files and have created the `patents.sq3` database file. It will take a few minutes to complete & you should see that the `patents.sq3` file is about 645MBytes in size.

In [ ]:
# Temporary fix to CSEL version conflict issue
# %pip install --upgrade --user ipython-sql==0.5.0

In [2]:
%%bash
# make
rm patents.sq3
zcat < acite75_99.zip | sqlite3 patents.sq3 ".mode csv" ".import /dev/stdin citations"
zcat < apat63_99.zip | sqlite3 patents.sq3 ".mode csv" ".import /dev/stdin patents"
ls -l patents.sq3

-rw-r--r-- 1 jovyan users 647274496 Sep 11 03:36 patents.sq3


* This prints the first 5 values in both the files

In [7]:
%%bash
zcat < acite75_99.zip | head -5
zcat < apat63_99.zip | head -5

"CITING","CITED"
3858241,956203
3858241,1324234
3858241,3398406
3858241,3557384
"PATENT","GYEAR","GDATE","APPYEAR","COUNTRY","POSTATE","ASSIGNEE","ASSCODE","CLAIMS","NCLASS","CAT","SUBCAT","CMADE","CRECEIVE","RATIOCIT","GENERAL","ORIGINAL","FWDAPLAG","BCKGTLAG","SELFCTUB","SELFCTLB","SECDUPBD","SECDLWBD"
3070801,1963,1096,,"BE","",,1,,269,6,69,,1,,0,,,,,,,
3070802,1963,1096,,"US","TX",,1,,2,6,63,,0,,,,,,,,,
3070803,1963,1096,,"US","IL",,1,,2,6,63,,9,,0.3704,,,,,,,
3070804,1963,1096,,"US","OH",,1,,2,6,63,,3,,0.6667,,,,,,,


We'll use another "magic" to run SQL queries in notebook cells. The following will load the SQL extension and connect to the `patents.sq3` file.

In [1]:
%load_ext sql
%sql sqlite:///patents.sq3
%config SqlMagic.style = '_DEPRECATED_DEFAULT'


Following this, we can run individual SQL queries and see the result by putting `%%sql` at the front of a cell. If you don't have that, you'll be running Python code.

So, for example, we can examine our two raw database tables.

In [3]:
%%sql
select * from patents limit 5;

 * sqlite:///patents.sq3
Done.


PATENT,GYEAR,GDATE,APPYEAR,COUNTRY,POSTATE,ASSIGNEE,ASSCODE,CLAIMS,NCLASS,CAT,SUBCAT,CMADE,CRECEIVE,RATIOCIT,GENERAL,ORIGINAL,FWDAPLAG,BCKGTLAG,SELFCTUB,SELFCTLB,SECDUPBD,SECDLWBD
3070801,1963,1096,,BE,,,1,,269,6,69,,1,,0,,,,,,,
3070802,1963,1096,,US,TX,,1,,2,6,63,,0,,,,,,,,,
3070803,1963,1096,,US,IL,,1,,2,6,63,,9,,0.3704,,,,,,,
3070804,1963,1096,,US,OH,,1,,2,6,63,,3,,0.6667,,,,,,,
3070805,1963,1096,,US,CA,,1,,2,6,63,,1,,0,,,,,,,


In [4]:
%%sql
select * from citations limit 5;

 * sqlite:///patents.sq3
Done.


CITING,CITED
3858241,956203
3858241,1324234
3858241,3398406
3858241,3557384
3858241,3634889


In [5]:
%%sql
SELECT name FROM sqlite_master WHERE type='table';

 * sqlite:///patents.sq3
Done.


name
citations
patents


In [22]:
%%sql
PRAGMA table_info(citations);

 * sqlite:///patents.sq3
Done.


cid,name,type,notnull,dflt_value,pk
0,CITING,TEXT,0,None,0
1,CITED,TEXT,0,None,0


In [27]:
%%sql
PRAGMA table_info(patents);

 * sqlite:///patents.sq3
Done.


cid,name,type,notnull,dflt_value,pk
0,PATENT,TEXT,0,None,0
1,GYEAR,TEXT,0,None,0
2,GDATE,TEXT,0,None,0
3,APPYEAR,TEXT,0,None,0
4,COUNTRY,TEXT,0,None,0
5,POSTATE,TEXT,0,None,0
6,ASSIGNEE,TEXT,0,None,0
7,ASSCODE,TEXT,0,None,0
8,CLAIMS,TEXT,0,None,0
9,NCLASS,TEXT,0,None,0


If you want to create indexes over various fields, go ahead. It shouldn't affect the correctness of your results but may affect the performance.

## Steps to the full solution

In order to determine when a *cited* patent and a *citing* patent are from the same state,
we're going to need to produce a series of tables that combine information from the citations and the patents tables.

We can use a simple left-outer join (or just **JOIN**) to get the information for one or the other column in the citations table. For example, we can determine the state for *cited* patents using this join:

In [3]:
%%sql
SELECT CITED, patents.POSTATE as CITED_POSTATE, CITING
FROM citations JOIN patents
WHERE CITED==PATENT limit 5;

 * sqlite:///patents.sq3
Done.


CITED,CITED_POSTATE,CITING
3398406,FL,3858241
3557384,MA,3858241
3634889,OH,3858241
3319261,OH,3858242
3668705,WI,3858242


Then, you'll need to do the same for the `CITING` column as well. As mentioned earlier, you may want to review [using `select` in a where-clause or using multiple joins](https://dba.stackexchange.com/questions/33553/using-select-in-the-where-clause-of-another-select).

# Testing

- let's try finding the cited patents from CO first

In [6]:
%%sql
SELECT * FROM patents
JOIN citations ON citations.CITED=patents.PATENT AND POSTATE='CO' 
LIMIT 5;

 * sqlite:///patents.sq3
Done.


PATENT,GYEAR,GDATE,APPYEAR,COUNTRY,POSTATE,ASSIGNEE,ASSCODE,CLAIMS,NCLASS,CAT,SUBCAT,CMADE,CRECEIVE,RATIOCIT,GENERAL,ORIGINAL,FWDAPLAG,BCKGTLAG,SELFCTUB,SELFCTLB,SECDUPBD,SECDLWBD,CITING,CITED
3070815,1963,1096,,US,CO,,1,,7,5,59,,1,,0,,,,,,,,4136548,3070815
3070863,1963,1096,,US,CO,,1,,24,6,63,,1,,0,,,,,,,,5372510,3070863
3071018,1963,1096,,US,CO,,1,,476,5,53,,3,,0.6667,,,,,,,,4174812,3071018
3071018,1963,1096,,US,CO,,1,,476,5,53,,3,,0.6667,,,,,,,,4391156,3071018
3071018,1963,1096,,US,CO,,1,,476,5,53,,3,,0.6667,,,,,,,,4693134,3071018


In [7]:
%%sql
SELECT * FROM 
patents as p1 
JOIN citations ON p1.PATENT = citations.CITED 
JOIN patents as p2 ON p2.PATENT = citations.CITING
WHERE p1.POSTATE = 'CO' AND p2.POSTATE = 'CO'
LIMIT 5;

 * sqlite:///patents.sq3
Done.


PATENT,GYEAR,GDATE,APPYEAR,COUNTRY,POSTATE,ASSIGNEE,ASSCODE,CLAIMS,NCLASS,CAT,SUBCAT,CMADE,CRECEIVE,RATIOCIT,GENERAL,ORIGINAL,FWDAPLAG,BCKGTLAG,SELFCTUB,SELFCTLB,SECDUPBD,SECDLWBD,CITING,CITED,PATENT_1,GYEAR_1,GDATE_1,APPYEAR_1,COUNTRY_1,POSTATE_1,ASSIGNEE_1,ASSCODE_1,CLAIMS_1,NCLASS_1,CAT_1,SUBCAT_1,CMADE_1,CRECEIVE_1,RATIOCIT_1,GENERAL_1,ORIGINAL_1,FWDAPLAG_1,BCKGTLAG_1,SELFCTUB_1,SELFCTLB_1,SECDUPBD_1,SECDLWBD_1
3076514,1963,1131,,US,CO,,2,,175,6,64,,8,,0.5625,,,,,,,,3966369,3076514,3966369,1976,6024,1975,US,CO,172825,2,10,418,5,53,6,9,0.5,0.6667,0.4444,16,17,1,0.1667,1,0.1111
3079856,1963,1159,,US,CO,,1,,100,5,59,,11,,0,,,,,,,,5009155,3079856,5009155,1991,11435,1989,US,CO,,1,7,100,5,59,32,8,0.7188,0.2188,0.1626,4.125,30.875,,,,
3083004,1963,1180,,US,CO,,2,,269,6,69,,1,,0,,,,,,,,4848759,3083004,4848759,1989,10791,1988,US,CO,,1,16,269,6,69,8,0,0.375,,0,,34.75,,,,
3085489,1963,1201,,US,CO,,1,,454,5,59,,2,,0.5,,,,,,,,4330500,3085489,4330500,1982,8173,1981,US,CO,73175,2,10,264,5,51,16,4,0.5625,0.375,0.6667,11.75,18.6875,0,0,0,0
3085565,1963,1201,,US,CO,,2,,126,6,66,,8,,0.6563,,,,,,,,4415759,3085565,4415759,1983,8719,1981,US,CO,614525,2,57,136,4,45,26,3,0.9615,0.4444,0.688,8.6667,10.1538,0,0,0,0


In [5]:
%%sql
SELECT COUNT(*) FROM 
patents as p1 INNER JOIN citations ON p1.patent=citations.CITED 
INNER JOIN patents as p2 ON p2.patent=citations.CITING 
WHERE p1.POSTATE='CO' AND p2.POSTATE='CO';

 * sqlite:///patents.sq3
Done.


COUNT(*)
14499


In [12]:
%%sql
SELECT citations.CITED, citations.CITING, COUNT(*) as COUNT_CITING_PATENTS, 
p1.POSTATE FROM
patents as p1 JOIN citations 
ON p1.patent=citations.CITED 
JOIN patents as p2 ON p2.patent=citations.CITING
WHERE p1.POSTATE=p2.POSTATE
GROUP BY citations.CITING LIMIT 5;

 * sqlite:///patents.sq3
Done.


CITED,CITING,COUNT_CITING_PATENTS,POSTATE
3557384,3858241,1,MA
3755824,3858245,1,NY
3621837,3858247,2,CA
3418664,3858249,4,TX
3694825,3858251,1,WI


this is only for colorado citation 

In [4]:
%%sql
SELECT p.PATENT, p.POSTATE, 
       COALESCE(counts.co_state_citations, 0) AS CO_STATE_CITATIONS
FROM patents p
LEFT JOIN (
    SELECT c.CITING, COUNT(*) AS co_state_citations
    FROM citations c
    JOIN patents p1 ON c.CITED = p1.PATENT
    JOIN patents p2 ON c.CITING = p2.PATENT
    WHERE p1.POSTATE = p2.POSTATE 
      AND p1.POSTATE = 'CO'
    GROUP BY c.CITING
) counts ON p.PATENT = counts.CITING
WHERE p.POSTATE = 'CO'
ORDER BY CO_STATE_CITATIONS DESC
LIMIT 10;

 * sqlite:///patents.sq3
Done.


PATENT,POSTATE,CO_STATE_CITATIONS
5974007,CO,25
5878015,CO,24
5760986,CO,23
5869164,CO,23
5796703,CO,22
5828054,CO,22
5875158,CO,22
5390384,CO,21
5694267,CO,21
5729511,CO,21


this is for wo states CA and CO

In [5]:
%%sql
SELECT p.PATENT, p.POSTATE, 
       COALESCE(counts.co_state_citations, 0) AS CO_STATE_CITATIONS
FROM patents p
LEFT JOIN (
    SELECT c.CITING, COUNT(*) AS co_state_citations
    FROM citations c
    JOIN patents p1 ON c.CITED = p1.PATENT
    JOIN patents p2 ON c.CITING = p2.PATENT
    WHERE p1.POSTATE = p2.POSTATE 
      AND p1.POSTATE IN ('CO', 'CA')  -- Test with 3 states
    GROUP BY c.CITING
) counts ON p.PATENT = counts.CITING
WHERE p.POSTATE IN ('CO', 'CA')
ORDER BY CO_STATE_CITATIONS DESC
LIMIT 10;

 * sqlite:///patents.sq3
Done.


PATENT,POSTATE,CO_STATE_CITATIONS
5959466,CA,125
6008204,CA,100
5952345,CA,98
5958954,CA,96
5998655,CA,96
5936426,CA,94
5739256,CA,90
5913855,CA,90
5925042,CA,90
5951547,CA,90


## Your solution

Eneter your solution as a single SQL query below:

In [ ]:
%%sql
SELECT p.PATENT, p.POSTATE, p.GYEAR, p.ASSIGNEE,
       COALESCE(counts.co_state_citations, 0) AS CO_STATE_CITATIONS
FROM patents p
LEFT JOIN (
    SELECT c.CITING, COUNT(*) AS co_state_citations
    FROM citations c
    JOIN patents p1 ON c.CITED = p1.PATENT
    JOIN patents p2 ON c.CITING = p2.PATENT
    WHERE p1.POSTATE = p2.POSTATE 
      AND p1.POSTATE IS NOT NULL
      AND p2.POSTATE IS NOT NULL
    GROUP BY c.CITING
) counts ON p.PATENT = counts.CITING
ORDER BY CO_STATE_CITATIONS DESC;

 * sqlite:///patents.sq3
Done.


In [2]:
%%sql
SELECT 
    p.*, 
    c.CITING AS PATENT_1,
    COUNT(*) AS co_cited_count
FROM citations c
JOIN (
    SELECT * 
    FROM patents 
    WHERE POSTATE IS NOT NULL 
      AND TRIM(POSTATE) <> ''
) p_citing 
    ON c.CITING = p_citing.PATENT
JOIN (
    SELECT * 
    FROM patents 
    WHERE POSTATE IS NOT NULL 
      AND TRIM(POSTATE) <> ''
) p_cited 
    ON c.CITED = p_cited.PATENT
JOIN patents p 
    ON p.PATENT = c.CITING
WHERE p_citing.POSTATE = p_cited.POSTATE
GROUP BY c.CITING
ORDER BY co_cited_count DESC
LIMIT 13;

 * sqlite:///patents.sq3
Done.


PATENT,GYEAR,GDATE,APPYEAR,COUNTRY,POSTATE,ASSIGNEE,ASSCODE,CLAIMS,NCLASS,CAT,SUBCAT,CMADE,CRECEIVE,RATIOCIT,GENERAL,ORIGINAL,FWDAPLAG,BCKGTLAG,SELFCTUB,SELFCTLB,SECDUPBD,SECDLWBD,PATENT_1,co_cited_count
5959466,1999,14515,1997,US,CA,5310,2,,326,4,46,159,0,1,,0.6186,,4.8868,0.0455,0.044,,,5959466,125
5983822,1999,14564,1998,US,TX,569900,2,,114,5,55,200,0,0.995,,0.7201,,12.45,0,0,,,5983822,103
6008204,1999,14606,1998,US,CA,749584,2,,514,3,31,121,0,1,,0.7415,,5,0.0085,0.0083,,,6008204,100
5952345,1999,14501,1997,US,CA,749584,2,,514,3,31,118,0,1,,0.7442,,5.1102,0,0,,,5952345,98
5998655,1999,14585,1998,US,CA,,1,,560,1,14,114,0,1,,0.7387,,5.1667,,,,,5998655,96
5958954,1999,14515,1997,US,CA,749584,2,,514,3,31,116,0,1,,0.7397,,5.181,0,0,,,5958954,96
5936426,1999,14466,1997,US,CA,5310,2,,326,4,46,178,0,1,,0.58,,11.2303,0.0765,0.073,,,5936426,94
5980517,1999,14557,1998,US,CA,733846,2,,606,3,32,241,0,1,,0.7394,,8.3776,0,0,,,5980517,90
5978329,1999,14550,1995,US,CA,148925,2,,369,2,24,145,0,1,,0.5449,,12.9241,0.4196,0.4138,,,5978329,90
5951547,1999,14501,1997,US,CA,733846,2,,606,3,32,242,0,1,,0.7382,,8.3471,0,0,,,5951547,90
